# Experimentation Only

In [1]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm, tqdm_notebook
%matplotlib inline

#For Prediction
from sklearn import metrics
from sklearn import preprocessing, cross_validation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.layers import Dense, Dropout, Input, LSTM, Reshape, Flatten
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Read Datasets 
research_data = pd.read_csv('Datasets/BlockChain_Train_csv.csv')
research_data.head()

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/JPY,USD/CHF,USD/KRW,USD/GBP,EUR/KRW,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate
0,3-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,90.73,1.06,1269.75,0.69,1774.86,6.83,0,0.0,0.08,0.15
1,5-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,92.08,1.08,1326.15,0.69,1843.95,6.82,0,0.0,0.11,0.15
2,7-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,93.79,1.12,1305.55,0.67,1763.60,6.84,0,0.0,0.11,0.47
3,9-Jan-09,0.0,0.0,750.0,0.0,0.0,0.0,0,1.0,0.0,...,91.24,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56
4,11-Jan-09,0.0,0.0,7600.0,0.0,0.0,0.0,0,1.0,0.0,...,91.24,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56


In [4]:
research_data.tail()

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/JPY,USD/CHF,USD/KRW,USD/GBP,EUR/KRW,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate
1768,9-Sep-18,6286.43,291261346.9,17259100.0,1.080000e+11,181951.87,0.62,0,1158.97,7.03,...,111.01,0.97,1128.55,0.77,1305.90,6.84,54,0.54,1.92,2.10
1769,11-Sep-18,6296.32,290980772.8,17262550.0,1.090000e+11,182206.04,0.94,0,1684.90,10.53,...,111.59,0.97,1125.80,0.77,1304.82,6.88,65,0.65,1.92,2.12
1770,13-Sep-18,6450.18,321882304.6,17266500.0,1.110000e+11,182446.38,0.82,0,1582.49,8.08,...,111.86,0.97,1120.89,0.76,1309.12,6.86,65,0.65,1.92,2.10
1771,15-Sep-18,6518.66,360631436.3,17269900.0,1.130000e+11,182696.55,0.79,0,1507.65,9.03,...,112.06,0.97,1121.43,0.77,1304.73,6.87,50,0.50,1.92,2.10
1772,17-Sep-18,6400.60,181083665.0,17273325.0,1.110000e+11,182906.28,0.91,0,1778.90,9.37,...,111.98,0.96,1124.70,0.76,1314.71,6.88,58,0.58,1.92,2.10


In [5]:
TIME_STEPS = 5
BATCH_SIZE_OUT = 32 # 16 was better
LR = 0.005
# epochs = 150
EPOCHS = 50

In [6]:
# Normalize and convert DataFrame into ndArray
# train_cols = ["Open","High","Low","Close","Volume USD","Popularity_Normalized"]
#train_cols = ["Close","TransactionsPerBlock","USD_Exchange_Trade_Volume","BlockSize_d","Total_Number_of_Transactions_d","Estimated_Transaction_Value_USD"]
train_cols = ["Close"]

df_train, df_test = train_test_split(research_data, train_size=0.8, test_size=0.2, shuffle=False)
print("Train and Test size", len(df_train), len(df_test))

# scale the feature MinMax, build array
x = df_train.loc[:,train_cols].values
min_max_scaler = MinMaxScaler()
x_train_out = min_max_scaler.fit_transform(x)
x_test_out = min_max_scaler.transform(df_test.loc[:,train_cols])


Train and Test size 1418 355


In [7]:
def build_timeseries(mat, y_col_index):
    # y_col_index is the index of column that would act as output column
    # total number of time-series samples would be len(mat) - TIME_STEPS
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]
    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))
    
#     for i in tqdm_notebook(range(dim_0)):
#         x[i] = mat[i:TIME_STEPS+i]
#         y[i] = mat[TIME_STEPS+i, y_col_index]
#     print("length of time-series i/o",x.shape,y.shape)
    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]
    #print("length of time-series i/o",x.shape,y.shape)
    return x, y

In [8]:
def trim_dataset(mat, batch_size):
    """
    trims dataset to a size that's divisible by BATCH_SIZE_OUT
    """
    no_of_rows_drop = mat.shape[0]%batch_size
    if(no_of_rows_drop > 0):
        return mat[:-no_of_rows_drop]
    else:
        return mat

In [9]:
def load_data(x_train, x_test, batch_size):
    x_t, y_t = build_timeseries(x_train, 0) # index of output (3)
    x_t = trim_dataset(x_t, batch_size)
    y_t = trim_dataset(y_t, batch_size)
    x_temp, y_temp = build_timeseries(x_test, 0) # index of output (3)
    x_val, x_test_t = np.split(trim_dataset(x_temp, batch_size),2)
    y_val, y_test_t = np.split(trim_dataset(y_temp, batch_size),2)
    
    return [x_t, y_t, x_val, x_test_t, y_val, y_test_t]

In [10]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [11]:
def create_generator():
#     generator=Sequential()
#     generator.add(Dense(units=128,input_dim=8))
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(units=256))
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(units=512))
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(units=8, activation='tanh'))
    
#     #generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer(), metrics=['accuracy'])
#     generator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 6 below = x_t.shape[2]
    generator = Sequential()
    #generator.add(LSTM(16, batch_input_shape=(BATCH_SIZE_OUT, TIME_STEPS, 6), dropout=0.0, recurrent_dropout=0.0, stateful=True, kernel_initializer='random_uniform', return_sequences=False)) 
    # stateful = true breaks the model (https://github.com/keras-team/keras/issues/9385)
    generator.add(LSTM(16, batch_input_shape=(BATCH_SIZE_OUT, TIME_STEPS, 1), dropout=0.0, recurrent_dropout=0.0, kernel_initializer='random_uniform', return_sequences=False)) 
    generator.add(Dense(8,activation='relu'))
    generator.add(Dropout(0.7))
    #generator.add(LSTM(8, return_sequences=False))
    generator.add(Dense(1,activation='sigmoid'))
    #generator.add(Reshape((5, 6)))
    optimizer = RMSprop(lr=LR, decay=3e-8)
    #generator.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    generator.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])


    return generator
g=create_generator()
g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (32, 16)                  1152      
_________________________________________________________________
dense_1 (Dense)              (32, 8)                   136       
_________________________________________________________________
dropout_1 (Dropout)          (32, 8)                   0         
_________________________________________________________________
dense_2 (Dense)              (32, 1)                   9         
Total params: 1,297
Trainable params: 1,297
Non-trainable params: 0
_________________________________________________________________


In [12]:
def create_discriminator():
    optimizer = RMSprop(lr=0.0002, decay=6e-8)

    discriminator=Sequential()
    #discriminator.add(Dense(units=512, batch_input_shape=(BATCH_SIZE_OUT, TIME_STEPS, 1)))
    discriminator.add(Dense(units=512, batch_input_shape=(155, TIME_STEPS, 1)))
    
    #discriminator.add(Dense(units=512,input_dim=6))
    #discriminator.add(Dense(units=512,input_shape=(BATCH_SIZE_OUT, TIME_STEPS, 6)))
    #discriminator.add(Dense(units=512,input_shape=(TIME_STEPS, 6)))
    #discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))
    
    #discriminator.add(Reshape((5, 1)))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#     discriminator = Sequential()
#     discriminator.add(LSTM(16, batch_input_shape=(BATCH_SIZE_OUT, TIME_STEPS, 6), dropout=0.0, recurrent_dropout=0.0, kernel_initializer='random_uniform', return_sequences=False)) 
#     discriminator.add(Dense(8,activation='relu'))
#     discriminator.add(Dropout(0.7))
#     discriminator.add(Dense(1,activation='sigmoid'))

#     discriminator.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    
    return discriminator
d =create_discriminator()
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (155, 5, 512)             1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (155, 5, 512)             0         
_________________________________________________________________
dropout_2 (Dropout)          (155, 5, 512)             0         
_________________________________________________________________
dense_4 (Dense)              (155, 5, 256)             131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (155, 5, 256)             0         
_________________________________________________________________
dropout_3 (Dropout)          (155, 5, 256)             0         
_________________________________________________________________
dense_5 (Dense)              (155, 5, 128)             32896     
__________

In [13]:
def create_gan(discriminator, generator):
    optimizer = RMSprop(lr=0.0001, decay=3e-8)
    discriminator.trainable=False
    gan_input = Input(shape=(TIME_STEPS, 1)) # x_t.shape[2] = 6
    #Input(shape=(None, latent_dim))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return gan
gan = create_gan(d,g)
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 1)              0         
_________________________________________________________________
sequential_1 (Sequential)    multiple                  1297      
_________________________________________________________________
sequential_2 (Sequential)    multiple                  165377    
Total params: 166,674
Trainable params: 1,297
Non-trainable params: 165,377
_________________________________________________________________


In [14]:
def create_gan2(d, g):
    optimizer = RMSprop(lr=0.0001, decay=3e-8)
    AM = Sequential()
    AM.add(g)
    AM.add(d)
    AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
        metrics=['accuracy'])
    return AM
#gan2 = create_gan2(d,g)
#gan2.summary()

In [15]:
import keras.backend as K

adversarial_loss = []
discrimator_loss = []
gen_samples = []

#K.clear_session()
generator = create_generator()
discriminator = create_discriminator()
gan = create_gan(discriminator, generator)

def training(epochs=1, batch_size=128):
    global adversarial_loss
    global discrimator_loss
    global gen_samples
    
    global generator
    global discriminator
    global gan
    
    #Loading the data
    #(X_train, y_train, X_test, y_test) = load_data()
    (X_train, y_train, X_val, X_test, y_val, y_test) = load_data(x_train_out, x_test_out, BATCH_SIZE_OUT)
    batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    # generator = create_generator()
    # discriminator = create_discriminator()
    # gan = create_gan(discriminator, generator)
    #gan = create_gan2(discriminator, generator)
    
    for e in range(1, epochs+1 ):
        print("Epoch %d" %e)
        for i in tqdm(range(batch_size)):
            #generate  random noise as an input  to  initialize the  generator
            #noise = np.random.normal(0,1, [BATCH_SIZE_OUT, TIME_STEPS, X_train.shape[2]])
            noise = np.float32(np.random.normal(size=[BATCH_SIZE_OUT, TIME_STEPS, X_train.shape[2]]))
            
            # Generate fake prices from noised input
            generated_prices = generator.predict(noise)
            
            # Get a random set of real prices
            prices_batch = X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
            
            dim_0 = generated_prices.shape[0] - TIME_STEPS
            dim_1 = generated_prices.shape[1]
            gen_prices = np.zeros((dim_0, TIME_STEPS, 1))

            #print('Shapes 1')
            #print(noise.shape)
            #print(prices_batch.shape)
            #print(generated_prices.shape)     
            #print(gen_prices.shape)
            #print(prices_batch)
            #print(generated_prices)
            
            #Construct different batches of real and fake data 
            X = np.concatenate([prices_batch, gen_prices])
            
            # Labels for generated and real data
            #y_dis = np.zeros(2*batch_size)
            #y_dis = np.zeros(shape=(2*BATCH_SIZE_OUT, TIME_STEPS, X_train.shape[2]))
            y_dis = np.zeros(shape=(X.shape[0], TIME_STEPS, X_train.shape[2]))
            #y_dis[:batch_size]=0.9
            y_dis[:int((X.shape[0]/2))]=0.9
            #y_dis[0,:,batch_size]=0.9
            
            #print('Shapes 2')
            #print(X.shape)
            #print(y_dis.shape)
            #print(y_dis)
            
            #Pre train discriminator on fake and real data before starting the gan. 
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            #noise = np.random.normal(0,1, [batch_size, 8])
            #y_gen = np.ones(batch_size)
            noise = np.float32(np.random.normal(size=[BATCH_SIZE_OUT, TIME_STEPS, X_train.shape[2]]))
            #y_gen = np.ones(shape=(noise.shape[0], TIME_STEPS, X_train.shape[2]))
            y_gen = np.ones(BATCH_SIZE_OUT)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #print('Shapes 3')
            #print(noise.shape)
            #print(y_gen.shape)
            
            #training the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            a_loss = gan.train_on_batch(noise, y_gen)
            
            if i == (batch_size-1):
                log_mesg = "%d: [D loss: %f, acc: %f]" % (e, d_loss[0], d_loss[1])
                log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
                print(log_mesg)
                
            discrimator_loss.append(d_loss)
            adversarial_loss.append(a_loss)
            
        if e == 1 or e % 5 == 0:
            # generate synthetic dataset
            gen_samples_set = []
            for batch_idx in range(int(len(X_train) / batch_size)):
            # for batch_idx in range(int(len(X_train) / 100)):
                # noise= np.random.normal(loc=0, scale=1, size=[100, 100])
                #noise= np.random.normal(0,1, [batch_size, 8])
                noise = np.float32(np.random.normal(size=[BATCH_SIZE_OUT, TIME_STEPS, X_train.shape[2]]))
                generated_data = generator.predict(noise)
                gen_samples_set.append(generated_data)

            gen_samples = np.vstack(gen_samples_set)

In [16]:
#training(400,128)
training(10,128)

  0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1


  2%|▏         | 2/128 [00:00<00:06, 19.43it/s]

1: [D loss: 0.670339, acc: 0.358710]  [A loss: 0.562025, acc: 1.000000]
Epoch 2


  2%|▏         | 2/128 [00:00<00:06, 18.49it/s]

2: [D loss: 0.681308, acc: 0.374194]  [A loss: 0.612188, acc: 1.000000]
Epoch 3


  2%|▏         | 2/128 [00:00<00:08, 14.17it/s]

3: [D loss: 0.689973, acc: 0.325161]  [A loss: 0.557811, acc: 1.000000]
Epoch 4


  2%|▏         | 2/128 [00:00<00:08, 14.83it/s]

4: [D loss: 0.684558, acc: 0.357419]  [A loss: 0.575221, acc: 1.000000]
Epoch 5


  2%|▏         | 2/128 [00:00<00:06, 18.67it/s]

5: [D loss: 0.677783, acc: 0.339355]  [A loss: 0.578386, acc: 1.000000]
Epoch 6


  2%|▏         | 2/128 [00:00<00:06, 19.40it/s]

6: [D loss: 0.679814, acc: 0.332903]  [A loss: 0.576299, acc: 0.968750]
Epoch 7


  2%|▏         | 3/128 [00:00<00:05, 21.05it/s]

7: [D loss: 0.668031, acc: 0.341935]  [A loss: 0.545777, acc: 0.968750]
Epoch 8


  2%|▏         | 2/128 [00:00<00:06, 18.22it/s]

8: [D loss: 0.681879, acc: 0.316129]  [A loss: 0.567335, acc: 1.000000]
Epoch 9


  2%|▏         | 2/128 [00:00<00:07, 16.13it/s]

9: [D loss: 0.653338, acc: 0.309677]  [A loss: 0.514272, acc: 0.937500]
Epoch 10


100%|██████████| 128/128 [00:06<00:00, 22.80it/s]

10: [D loss: 0.652449, acc: 0.361290]  [A loss: 0.439776, acc: 1.000000]


In [17]:
# generated_data = np.transpose(gen_samples)
# #print(gen_samples)
# #print(generated_data)
# #plot the log-returns
# gen_ind = 1 # change in function price as well
# pd.DataFrame(gen_samples).plot()

In [18]:
def get_2D_array_values(arr, idx):
    arr_map = []
    for i in range(len(arr)):
        arr_map.append(arr[i][idx])
    
    return arr_map

Text(0,0.5,'Loss')

In [19]:
adversarial_loss_map = get_2D_array_values(adversarial_loss, 0)
discrimator_loss_map = get_2D_array_values(discrimator_loss, 0)

ax = pd.DataFrame(
    {
        'Adversarial Loss': adversarial_loss_map,
        'Discriminative Loss': discrimator_loss_map,
    }
).plot(title='Training loss', logy=True)
ax.set_xlabel("Training iterations")
ax.set_ylabel("Loss")

Text(0,0.5,'Accuracy')

In [20]:
adversarial_acc_map = get_2D_array_values(adversarial_loss, 1)
discrimator_acc_map = get_2D_array_values(discrimator_loss, 1)

ax_acc = pd.DataFrame(
    {
        'Adversarial Accuracy': adversarial_acc_map,
        'Discriminative Accuracy': discrimator_acc_map,
    }
).plot(title='Training loss', logy=True)
ax_acc.set_xlabel("Training iterations")
ax_acc.set_ylabel("Accuracy")

In [23]:
#(X_train2, X_test2, y_train2, y_test2) = load_data_with_test2()
(X_train2, y_train2, X_val2, X_test2, y_val2, y_test2) = load_data(x_train_out, x_test_out, BATCH_SIZE_OUT)

print(x_train_out.shape)
print(x_test_out.shape)

print(X_train2.shape)
print(X_test2.shape)
print(X_val2.shape)

prediction_gan = generator.predict(X_test2)

# print("The Explained Variance: %.2f" % prediction_gan.score(X_test2, y_test2))  
print("The Explained Variance: %.2f" % metrics.explained_variance_score(y_test2, prediction_gan))  
print("The Mean Absolute Error: %.2f" % metrics.mean_absolute_error(y_test2, prediction_gan))  
#print("The Median Absolute Error: %.2f" % metrics.median_absolute_error(y_test2, prediction_gan)) 
print("The Mean Squared Error: %.2f" % metrics.mean_squared_error(y_test2, prediction_gan)) 
print("The Root Mean Squared Error: %.2f" % (np.sqrt(metrics.mean_squared_error(y_test2, prediction_gan))))

#generator.predict(X_train)[0]

(1418, 1)
(355, 1)
(1408, 5, 1)
(160, 5, 1)
(160, 5, 1)
The Explained Variance: 0.00
The Mean Absolute Error: 6.53
The Mean Squared Error: 51.20
The Root Mean Squared Error: 7.16
